# OHBM 2025 IBMA with NeuroVault tutorial

## IBMA: Image-based meta-analysis

IBMA, considered the gold standard of neuroimaging meta-analysis (Salimi-Khorshidi et al., 2009; Salo et al., 2023), consists of aggregating results from group-level, whole-brain statistical maps from individually conducted functional magnetic resonance imaging (fMRI) studies. IBMA outperforms other popular meta-analysis methods, such as coordinate-based meta-analysis (CBMA). IBMA methods use whole-brain statistics; thus, all existing voxel-wise statistical methods are available to analyze subject-level data within studies (Lazar et al., 2002). IBMA is known to produce richer and more detailed results, with additional brain structures that are often absent from CBMA results. IBMA also has greater power; thus, one could potentially achieve similar or even better results with a small fraction of studies generally required in CBMA. In addition, when both the parameters and variance estimates are available, hierarchical mixed effect models can be used to account for both within- and between-study variance (Salimi-Khorshidi et al., 2009).

## Tools

### NiMARE

![NiMARE banner](images/nimare_banner.png)

[NiMARE](https://nimare.readthedocs.io/en/latest/) is a Python library for performing neuroimaging meta-analyses and related analyses, like automated annotation and functional decoding. The goal of NiMARE is to centralize and standardize implementations of common meta-analytic tools, so that researchers can use whatever tool is most appropriate for a given research question.

### NeuroVault

![NeuroVault logo](images/neurovault-logo.svg)

[NeuroVault](https://neurovault.org) is a web-based repository of fMRI statistical maps from neuroimaging studies (Gorgolewski et al., 2015). The brain maps are grouped in collections that are created and updated voluntarily. This repository can be explored and downloaded with the help of an API, which is supported by some Python neuroimaging tools (e.g., Nilearn and NiMARE).

### Cognitive Atlas

![CogAt logo](images/cogat-logo.png)

[Cognitive Atlas](https://www.cognitiveatlas.org/) (Poldrack et al., 2011) is an online repository of cumulative knowledge from experienced researchers from the psychology, cognitive science, and neuroscience fields. The repository currently offers two knowledge bases: 907 cognitive concepts and 841 tasks with definitions and properties. Cognitive concepts contain relationships with other concepts and tasks, with the goal of establishing a map between mental processes and brain function. It provides an API to download the database, which is also integrated into NiMARE.

## Goals for this tutorial

1. Download data from NeuroVault
2. Identify usable Neurovault images for IBMA
3. Learn to identify outliers
4. Use NiMARE to run IBMA
5. Interpret IBMA results


In [ ]:
import tarfile
import os.path as op
import warnings
import logging
import os

import pandas as pd
from nimare.reports.base import run_reports
from nimare.workflows import IBMAWorkflow
import requests
from nimare.meta.ibma import Stouffers
import matplotlib.pyplot as plt
from nilearn.plotting import plot_stat_map

from utils import download_images, convert_to_nimare_dataset
from outliers import _rm_nonstat_maps, _rm_extreme_maps, _rm_duplicates_maps

warnings.filterwarnings("ignore")
logging.getLogger("nimare").setLevel(logging.ERROR)

In [ ]:
data_dir = op.abspath("../data")
filename = "november_2022"
nv_data_dir = op.join(data_dir, filename)
image_dir = op.join(data_dir, "nv_images")
report_dir = op.join(data_dir, "report")
report_clean_dir = op.join(data_dir, "report_clean")

os.makedirs(data_dir, exist_ok=True)

Download Neurovault data

To explore the NeuroVault database, we created an SQL query and exported the database contents to human-readable tables while filtering sensitive user information. This provided sufficient metadata from all collections and images to investigate the entire database without downloading the files. The images identified as usable for IBMA (see the following section on the image selection framework) were downloaded along with their metadata and converted to a NiMARE Dataset object to leverage existing IBMA methods implemented in NiMARE.

**Note**: The following three cells, which download the NeuroVault data tables, need to be run only once in this tutorial.


In [ ]:
github_url = (
    f"https://raw.githubusercontent.com/NeuroVault/nv-data/master/{filename}.tar.gz"
)
response = requests.get(github_url)

file_path = op.join(data_dir, f"{filename}.tar.gz")

In [ ]:
with open(file_path, "wb") as f:
    f.write(response.content)

In [ ]:
with tarfile.open(file_path, "r:gz") as tar:
    tar.extractall(path=data_dir)

In [ ]:
sorted(os.listdir(nv_data_dir))

The files available are the following:

- `django_content_type.csv`: No relevant
- `statmaps_atlas.csv`: Contains some atlases available in NeuroVaults
- `statmaps_basecollectionitem.csv`: Contains the image IDs, name, description, and the associated collection ID
- `statmaps_cognitiveatlascontrast.csv`: Contains the contrast names and IDs from Cognitive Atlas
- `statmaps_cognitiveatlastask.csv`: Contains the task names and IDs from Cognitive Atlas
- `statmaps_collection.csv`: Contains the collection-level metadata, including name, DOI, authors, etc.
- `statmaps_collection_communities.csv`: Contains the communities linked to a collection
- `statmaps_collection_contributors.csv`: Contains the user ID linked to a collection
- `statmaps_community.csv`: Contains the information about communities in NeuroVault, such as their label and description
- `statmaps_image.csv`: Contains image file name and demographic information linked to the image
- `statmaps_statisticmap.csv`: Contains image metadata such as map type, modality, task, analysis level, etc


To identify images usable for IBMA, first get the image's ID (`id`) and name (`name`), as well as their affiliated collection ID (`collection_id`).


In [ ]:
# Get images and collection IDs
cmeta_cols = ["id", "collection_id", "name"]
image_colection = pd.read_csv(
    op.join(nv_data_dir, "statmaps_basecollectionitem.csv"),
    usecols=cmeta_cols,
)

Second, from `statmaps_image.csv` we get the file name, which will use later to download the images using the NuroVault API. In this database `basecollectionitem_ptr_id` refers to the image ID `id`.


In [ ]:
# Get path to images in NeuroVault
image_cols = ["file", "basecollectionitem_ptr_id"]
image = pd.read_csv(
    op.join(nv_data_dir, "statmaps_image.csv"),
    usecols=image_cols,
)
image = image.rename(columns={"basecollectionitem_ptr_id": "id"})

Third, some relevant images and metadata are extracted from `statmaps_statisticmap.csv`.

The selected metadata included:

- `image_ptr_id`: Image ID in NeuroVault

- `map_type`: Type of statistic that is the basis of the inference

  - Possible cases: `T map`, `Z map`, `F map`, `Chi squared map`, `P map (given null hypothesis)`, `1-P map ("inverted" probability)`, `multivariate-beta map`, `Beta map`, `ROI/mask`, `parcellation`, `anatomical`, `variance`, `Other`

- `modality`: Brain imaging procedure that was used to acquire the data

  - Possible cases: `fMRI-BOLD`, `fMRI-CBF`, `fMRI-CBV`, `Diffusion MRI`, `Structural MRI`, `PET FDG`, `PET [150]-water`, `MEG`, `EEG`, `Other`

- `analysis_level`: What level of summary data was used as the input to this analysis?

  - Possible cases: `single-subject`, `group`, `meta-analysis`, `other`

- `number_of_subjects`: Number of subjects used to generate this map

  - Integer values ranging from 1 to N

- `is_thresholded`: Whether the map is thresholded or not

  - Possible cases: `True`, `False`

- `brain_coverage`: Percentage of brain coverage

- `not_mni`: Whether the image is in the MNI space or not

  - Possible cases: `True`, `False`

- `cognitive_paradigm_cogatlas_id`: Task performed by the subjects in the scanner described using [Cognitive Atlas](https://www.cognitiveatlas.org/)

  **Note**: One can find the complete list of Cognitive Atlas tasks and their IDs in the file `statmaps_cognitiveatlastask.csv`, or in the Cognitive Atlas [website](https://www.cognitiveatlas.org/tasks), which are also accessible via [API](https://www.cognitiveatlas.org/api/v-alpha/task).


In [ ]:
# Get image metadata
imeta_cols = [
    "image_ptr_id",
    "map_type",
    "modality",
    "analysis_level",
    "number_of_subjects",
    "is_thresholded",
    "brain_coverage",
    "not_mni",
    "cognitive_paradigm_cogatlas_id",
]
image_meta = pd.read_csv(
    op.join(nv_data_dir, "statmaps_statisticmap.csv"),
    usecols=imeta_cols,
)
image_meta = image_meta.rename(columns={"image_ptr_id": "id"})

Finally, we combine the previously extracted information into a single Pandas DataFrame by merging on the image ID (`id`).


In [ ]:
image_merge = pd.merge(image_colection, image, how="left", on="id")
image_full_df = pd.merge(image_merge, image_meta, how="left", on="id")

# Select images for IBMA

Using the available metadata from the retrieved tables, we set different inclusion criteria for images to be considered for a meta-analysis. We focused on fMRI-BOLD images, as they are the most prevalent modality in NeuroVault. Note that the methods presented in this tutorial should work with other image modalities (e.g., PET, diffusion MRI, structural MRI). Still, only fMRI-BOLD had enough data in NeuroVault for meta-analyses. Then, we specifically chose images from group-level analyses. Additionally, we retained only images from studies with a sample size greater than ten subjects. Next, we selected images classified as T or Z statistics. Although best practices in meta-analysis suggest using meaningful units and incorporating uncertainty through standard errors, T/Z statistic maps are the most commonly shared images in NeuroVault (Maumet and Nichols, 2016). We discuss this further in the following sections. Following that, we retained unthresholded images that cover 40% of the brain and are in MNI space. Ultimately, we narrowed our selection to images associated with a Cognitive Atlas task.

For this tutorial, we use the n-back task (`tsk_4a57abb949bcd`) as it has been validated before (Peraza et al., 2025). Users are encouraged to investigate other domains.


In [ ]:
image_selected_df = image_full_df.query(
    'modality == "fMRI-BOLD"'
    ' & analysis_level == "G"'
    " & number_of_subjects > 10"
    ' & (map_type == "Z" | map_type == "T")'
    ' & is_thresholded == "f"'
    " & brain_coverage > 40"
    ' & not_mni == "f"'
    ' & cognitive_paradigm_cogatlas_id == "tsk_4a57abb949bcd"'
)

# Download images for IBMA

Next, we attempt to download the selected image from NeuroVault. To facilitate that process, we implemented a function named download images, which takes the image IDs from the previously created Python DataFrame and a path to a directory to download the images. Note that not all images can be downloaded; some of them belong to a private collection, while others have a corrupted file.

Given that some images will have the same image name (e.g., z*stats.nii.gz), we downloaded the images, adding a unique identifiable prefix composed of the collection ID and the image ID. THis is, `[col_id]-[img_id]*[image_name].nii.gz`.


In [ ]:
image_ids = image_selected_df["id"].unique()
image_usable_df = download_images(image_ids, image_dir)
image_df = pd.merge(image_selected_df, image_usable_df, on="id")

print(f"Usable images: {len(image_usable_df)}/{len(image_ids)}")

# Create NiMARE Dataset object for IBMA

Then, we create a NiMARE Dataset object in order to leverage image-based meta-analytic methods from NiMARE. In NiMARE, datasets are stored in a special `Dataset` class. The `Dataset` class stores the most relevant information as properties.

For users' convenience, we have implemented a function `convert_to_nimare_dataset` to facilitate that process. The function takes a Pandas DataFrame and returns a NiMARE Database object.

In addition, we use the `ImageTransformer`, which can generate images from other images, as long as the right images and metadata are available. In this case, we generate z-statistic images from t-statistic maps, leveraging the sample size information in the metadata.

For additional detail on creating a Dataset class, please refer to this example: https://nimare.readthedocs.io/en/stable/auto_examples/01_datasets/06_plot_dataset_json.html#sphx-glr-auto-examples-01-datasets-06-plot-dataset-json-py.


In [ ]:
dset = convert_to_nimare_dataset(image_df)

The full list of identifiers in the Dataset is located in `Dataset.ids`. Identifiers are composed of two parts- a study ID and a contrast ID. Within the Dataset, those two parts are separated with a `-`.


In [ ]:
print(dset.ids)

Most other information is stored in `pandas` DataFrames. The five DataFrame-based attributes are `Dataset.metadata`, `Dataset.coordinates`, `Dataset.images`, `Dataset.annotations`, and `Dataset.texts`.

Each DataFrame contains at least three columns: `study_id`, `contrast_id`, and `id`, which is the combined `study_id` and `contrast_id`.


In [ ]:
dset.metadata.head()

In [ ]:
dset.images.head()

# Initialize an IBMA workflow with NiMARE

Next, we leverage the available NiMARE workflows that implements several meta-analytic pipelines. Specifically, we use an IBMAWorkflow

The parameters to initialize the workflow are:

- `estimator`: Meta-analysis estimator. Default Stouffers

  Useful parameter in Stouffers:

  - `aggressive_mask`: By default, all image-based meta-analysis estimators adopt an aggressive masking strategy, in which any voxels with a value of zero in any of the input maps will be removed from the analysis. Setting `aggressive_mask=False` will instead run the analysis in bags of voxels that have a valid value across the same studies.

  - `use_sample_size`: Whether to use sample sizes for weights (i.e., "weighted Stouffer's") or not.

  - `normalize_contrast_weights`: Whether to use a number of contrasts per study to normalize the weights or not.

  **Note**: The current implementation in NiMARE adds a correction factor to account for repeated scans (i.e., multiple samples from the same study).

- `corrector`: Meta-analysis corrector. Default FDR

- `diagnostics`: Diagnostic method. Default Jackknife

- `voxel_thresh`: An optional voxel-level threshold that may be applied to the `target_image` of the diagnostic class to define clusters. Default 1.6

- `cluster_threshold`: Cluster size threshold. Default 10

- `output_dir`: Output directory in which to save results. Default None

- `n_cores`: Number of cores to use for parallelization. Default 1

  Here, parallelization only applies to the Jackknife analysis.


In [ ]:
workflow = IBMAWorkflow(
    estimator=Stouffers(
        aggressive_mask=True,
        use_sample_size=True,
        normalize_contrast_weights=True,
    ),
    corrector="fdr",
    diagnostics="jackknife",
    voxel_thresh=3.2,
    cluster_threshold=90,
    output_dir=None,
    n_cores=1,
)

# Apply workflow to the Dataset object

The fit method of an IBMA workflow class runs the following steps:

1. Runs a meta-analysis using the specified method (default: Stouffers)

2. Applies a corrector to the meta-analysis results (default: FDRCorrector, indep)

3. Generates cluster tables and runs diagnostics on the corrected results (default: Jackknife)


In [ ]:
results = workflow.fit(dset)

# Plot Results

The fit method of the IBMA workflow class returns a MetaResult object, where you can access the corrected results of the meta-analysis and diagnostics tables.

Corrected map:


In [ ]:
img = results.get_map("z_corr-FDR_method-indep")
plot_stat_map(
    img,
    cut_coords=4,
    display_mode="z",
    threshold=1.65,  # voxel_thresh p < .05, one-tailed
    cmap="RdBu_r",
    symmetric_cbar=True,
    vmax=4,
)
plt.show()

## Clusters table


In [ ]:
results.tables["z_corr-FDR_method-indep_tab-clust"].head()

## Diagnostic: Jackknife

The Jackknife analysis characterizes the relative contribution of each experiment in a meta-analysis to the resulting clusters by looping through experiments, calculating the Estimator's summary statistic for all experiments except the target experiment, dividing the resulting test summary statistics by the summary statistics from the original meta-analysis, and finally averaging the resulting proportion values across all voxels in each cluster.


In [ ]:
results.tables["z_corr-FDR_method-indep_diag-Jackknife_tab-counts"].head()

# Generate an HTML report of IBMA results

Finally, a NiMARE report is generated from the MetaResult. The `run_reports` function takes two arguments: (1) A [MetaResult](https://nimare.readthedocs.io/en/stable/generated/nimare.results.MetaResult.html#nimare.results.MetaResult) object produced by the IBMA workflow, and (2) a path to an output directory in which to save the report.


In [ ]:
run_reports(results, report_dir)

# Exclude spurious images

## Identify and exclude non-statistical maps

Even after applying the previous strict preliminary inclusion criteria, we still found plenty of wrongly annotated images, especially representing other image modalities and others with extreme values. Therefore, we developed an automatic heuristic selection to remove those spurious images from the meta-analysis. The heuristic selection consisted of two steps. First, we removed all images from collections that lacked a link to a publication. Also, images with a minimum Z value smaller than 1.96 (i.e., Z score for a 0.05 p-value) were removed as they potentially consisted of mislabeled correlation maps, inverted p-value maps, or did not contain statistically significant voxels. We also excluded images with a maximum Z score larger than 50. Although the number 50 is arbitrary, we wanted to detect images with an unusually large signal. For example, mislabeled BOLD or COPE (contrast of parameter estimates) images or others resulting from studies with a huge sample size. Additionally, using the image metadata, we analyzed the image and file name. We removed those containing keywords such as "ICA," "PCA," "PPI," "seed," "functional connectivity," “cope,” “tfce,” and "correlation," which represent modalities not of interest for the meta-analysis of the current work. Example: https://neurovault.org/images/31938/


In [ ]:
image_clean_df = _rm_nonstat_maps(image_df, verbose=1)
image_clean_df = _rm_extreme_maps(image_clean_df, verbose=1)

In [ ]:
dset_clean = convert_to_nimare_dataset(image_clean_df)

## Identify and exclude duplicate and inverted contrast images

It is quite common for NeuroVault users to upload inverted contrasts and duplicates. For example, one might find two images representing the same contrast (such as House > Face) but with the signs reversed (i.e., Face > House). Example of inverted contrasts in NeuroVault: [MAINFOOD_HCplus>HCmin](https://neurovault.org/images/123498/) and [MAINFOOD_HCmin>HCplus](https://neurovault.org/images/123499/). This creates problems for meta-analyses, as these images effectively cancel each other out when aggregated.

Additionally, it is typical for users to upload multiple images of the same contrast, differing only by the covariate used in the group-level analysis. These can be considered duplicates, especially when the covariate does not influence the final estimate. To identify duplicates, we utilize the correlation matrix of the input samples. Image pairs with a correlation close to 1 are considered duplicates, while those with a correlation close to -1 are labeled as inverted contrasts. From the identified duplicates, we randomly selected one image from each pair. For pairs of inverted contrasts, we choose the image with the bigger correlation relative to the median image.


In [ ]:
dset_clean = _rm_duplicates_maps(dset_clean, verbose=1)

Run the IBMA workflow on the clean dataset


In [ ]:
workflow_clean = IBMAWorkflow(
    estimator=Stouffers(
        aggressive_mask=True,
        use_sample_size=True,
        normalize_contrast_weights=True,
    ),
    corrector="fdr",
    diagnostics="jackknife",
    voxel_thresh=3.2,
    cluster_threshold=90,
    output_dir=None,
    n_cores=1,
)
results_clean = workflow_clean.fit(dset_clean)
run_reports(results_clean, report_clean_dir)

# Next steps

Now, is you turn to work


# References

For additional detail on the methods presented in this tutorial plase refer to our preprint: https://doi.org/10.1101/2025.03.06.641922

- Gorgolewski, K.J., Varoquaux, G., Rivera, G., Schwarz, Y., Ghosh, S.S., Maumet, C., Sochat, V.V., Nichols, T.E., Poldrack, R.A., Poline, J.-B., Yarkoni, T., Margulies, D.S., 2015. NeuroVault.org: a web-based repository for collecting and sharing unthresholded statistical maps of the human brain. Frontiers in Neuroinformatics 9, 8. https://doi.org/10.3389/fninf.2015.00008

- Lazar, N.A., Luna, B., Sweeney, J.A., Eddy, W.F., 2002. Combining Brains: A Survey of Methods for Statistical Pooling of Information. NeuroImage 16, 538–550. https://doi.org/10.1006/nimg.2002.1107

- Maumet, C., Nichols, T.E., 2016. Minimal Data Needed for Valid & Accurate Image-Based fMRI Meta-Analysis. https://doi.org/10.1101/048249

- Peraza, J.A., Kent, J.D., Blair, R.W., Poline, J.-B., Nichols, T.E., Vega, A. de la, Laird, A.R., 2025. Advancing image-based meta-analysis for fMRI: A framework for leveraging NeuroVault data. https://doi.org/10.1101/2025.03.06.641922

- Poldrack, R.A., Kittur, A., Kalar, D., Miller, E., Seppa, C., Gil, Y., Parker, D.S., Sabb, F.W., Bilder, R.M., 2011. The cognitive atlas: toward a knowledge foundation for cognitive neuroscience. Front Neuroinform 5, 17. https://doi.org/10.3389/fninf.2011.00017

- Salimi-Khorshidi, G., Smith, S.M., Keltner, J.R., Wager, T.D., Nichols, T.E., 2009. Meta-analysis of neuroimaging data: A comparison of image-based and coordinate-based pooling of studies. NeuroImage 45, 810–823. https://doi.org/10.1016/j.neuroimage.2008.12.039

- Salo, T., Yarkoni, T., Nichols, T.E., Poline, J.-B., Bilgel, M., Bottenhorn, K.L., Jarecka, D., Kent, J.D., Kimbler, A., Nielson, D.M., Oudyk, K.M., Peraza, J.A., Pérez, A., Reeders, P.C., Yanes, J.A., Laird, A.R., 2023. NiMARE: Neuroimaging Meta-Analysis Research Environment. Aperture Neuro 3, 1–32. https://doi.org/10.52294/001c.87681
